In [123]:
from time import sleep
from os import path
import googleapiclient.discovery
import json
import os

In [124]:
CHANNEL_YUTT = 'UCQJ6ESCWQotBwtJm0Ff_gyQ'
DEVELOPER_KEY = 'KEY'

In [125]:
def yt_search_list(channelId, **kwargs):
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.search().list(
        part="snippet",
        channelId=channelId,
        order='date',
        maxResults=50,
        **kwargs
    )
    response = request.execute()

    return response

In [126]:
def yt_videos_list(ids):
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.videos().list(
        part="snippet,contentDetails,topicDetails,statistics,liveStreamingDetails",
        maxResults=50,
        id=ids
    )
    response = request.execute()
    
    return response

In [127]:
with open('./videos_list_yutt.json') as f:
    videos_list_yutt = sorted(json.loads(f.read()), key=lambda x: x['snippet']['publishedAt'], reverse=True)

In [128]:
new_videos = yt_search_list(CHANNEL_YUTT, publishedAfter=search_list_yutt[0]['snippet']['publishedAt'])
assert new_videos['pageInfo']['totalResults'] <= 50

In [129]:
video_ids = set([v['id'] for v in videos_list_yutt if v['kind'] == 'youtube#video'])
new_video_ids = [v['id']['videoId'] for v in new_videos['items'] if not (v['id']['kind'] == 'youtube#video' and v['id']['videoId'] in video_ids)]

In [130]:
new_video_list = yt_videos_list(','.join(new_video_ids))
videos_list_yutt_updated = new_video_infos['items'] + videos_list_yutt
assert set([v['id'] for v in videos_list_yutt_updated if v['kind'] == 'youtube#video']) == (set(new_video_ids).union(video_ids))

In [134]:
with open('./videos_list_yutt.json', 'w') as f:
    f.write(json.dumps(videos_list_yutt_updated, indent=2))